### Model 1: Minimizing total swimmers time 

In [2]:
#Load packages
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays, Distributions
const GRB_ENV = Gurobi.Env(output_flag=0);

In [3]:
# Load data
swimmer_times = CSV.read("swimmer_event_times.csv", DataFrame)
swimmer_stats = CSV.read("swimmer_stats.csv", DataFrame)
individual_event_order = CSV.read("individual_event_order.csv", DataFrame)
relay_event_order = CSV.read("relay_event_order.csv", DataFrame)
opponent_times = CSV.read("opponent_times.csv", DataFrame)

Row,Opponent,Event,Opponent_Time
,String31,String31,Float64
1,Group 1,200 Medley Relay,102.62
2,Group 2,200 Medley Relay,104.83
3,"Bastone, Alexandra",1000 Free,603.44
4,"Wood Prince, Piper",1000 Free,622.34
5,"Cruz-Abrams, Robin",1000 Free,653.79
6,"McDonald, Kaylee",200 Free,111.89
7,"Foster, Payton",200 Free,114.7
8,"Hakonardottir, Kristin",200 Free,116.2
9,"Hamlin, Molly",100 Back,54.94


In [4]:
# Create dictionaries
best_times = Dict((row[1], row[2]) => row[3] for row in eachrow(swimmer_times))
mean_times = Dict((row[1], row[2]) => row[3] for row in eachrow(swimmer_stats))
std_dev_times = Dict((row[1], row[2]) => row[4] for row in eachrow(swimmer_stats))
individual_event_order_list = individual_event_order[:, 2]
relay_event_positions = Dict(row[2] => row[3:6] for row in eachrow(relay_event_order))
swimmer_list = unique(swimmer_times[:, 1])


31-element Vector{String31}:
 "Adler, Lauren"
 "Augustyn, Kate"
 "Beggs, Christina"
 "Bernard, Sarah"
 "Chen, Olivia"
 "Chun, Sydney"
 "Crane, Jessie"
 "Eppinger, Aria"
 "Feliz, Mary"
 "Griffin, Miri"
 "Kim, Jolie"
 "Kudela, Katie"
 "Levy, Lauren"
 ⋮
 "Shipps, Stella"
 "Simons, Kailey"
 "Smith, Sydney"
 "Swanson, Chloe"
 "Swartwood, Belise"
 "Tang, Natalie"
 "Turvey, Alexandra"
 "Yang, Iris"
 "Yang, Rachel"
 "Yao, Katherine"
 "Zhao, Kathy"
 "Zhu, Emma"

In [5]:
# Parameters
swimmer_count = 31
num_individual_events = 12
num_relay_events = 2 # events 1 and 14
num_relay_positions = 4 # number of positions in each relay
num_groups = 3
num_places = 6 # 6 different places to finish a race

penalty_time = 99999  # A large penalty value to discourage invalid assignments

99999

In [6]:
model_1 = Model(() -> Gurobi.Optimizer(GRB_ENV))
#variables 
@variable(model_1, x[1:swimmer_count, 1:num_individual_events], Bin) #decision variable of if swimmer competes in event 
@variable(model_1, y[1:swimmer_count, 1:num_relay_positions, 1:num_relay_events, 1:num_groups], Bin) #decicsion variable if swimmer is 
#in the position for relay event in a certain group 

@objective(model_1, Min, 
    # Individual events
    sum(get(best_times, (swimmer_list[s], individual_event_order_list[e]), penalty_time) * x[s, e]
        for s in 1:swimmer_count, e in 1:num_individual_events) +
    
    # Relay events
    sum(get(best_times, (swimmer_list[s], relay_event_positions[relay_event_order[r, 2]][i]), penalty_time) * y[s, i, r, g]
        for s in 1:swimmer_count, i in 1:num_relay_positions, r in 1:num_relay_events, g in 1:num_groups))

# Each swimmer swims 3 individual events
@constraint(model_1, [s in 1:swimmer_count], 
    sum(x[s, e] for e in 1:num_individual_events) <= 3)

# Each individual event we have only one swimmer
@constraint(model_1, [e in 1:num_individual_events], 
    sum(x[s, e] for s in 1:swimmer_count) >= 1) 

# 4 persons per relay group
@constraint(model_1, [r in 1:num_relay_events, g in 1:num_groups], 
    sum(y[s, i, r, g] for s in 1:swimmer_count, i in 1:num_relay_positions) == 4)

# Can't swim back-to-back events
for s in 1:swimmer_count
    for e in 1:num_individual_events-1
        @constraint(model_1, x[s, e] + x[s, e+1] <= 1)
    end
end

# Each swimmer can only swim once in first relay
@constraint(model_1, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# Each swimmer can only swim once in last relay
@constraint(model_1, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# If swim in first relay, can't swim first individual event
@constraint(model_1, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) + x[s, 1] <= 1)

# If swim in last relay, can't swim last individual event
@constraint(model_1, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) + x[s, num_individual_events] <= 1)

# Exclude the pairs with missing times
#@constraint(model_1, [s in 1:swimmer_count, e in 1:num_individual_events], 
 #   x[s, e] <= haskey(best_times, (swimmer_list[s], individual_event_order_list[e])))

# Each swimmer can only swim in one position in a group
@constraint(model_1, [r in 1:num_relay_events, g in 1:num_groups, i in 1:num_relay_positions], 
    sum(y[s, i, r, g] for s in 1:swimmer_count) == 1)

#optimizing 
optimize!(model_1)

if termination_status(model_1) == MOI.OPTIMAL
    println("Optimal solution found")
else
    println("No optimal solution found")
end

println("Total swimming time: ", objective_value(model_1))

Optimal solution found
Total swimming time: 2412.7999999999997


In [7]:
#printing out what events the swimmer are swimming and what times 
for s in 1:swimmer_count
    for e in 1:num_individual_events
        if value(x[s, e]) > 0.5
            swimmer = swimmer_list[s]
            event = individual_event_order[e, :event]
            best_time = get(best_times, (swimmer, event), "N/A")
            println("Swimmer $swimmer assigned to event $event (Best time: $best_time)")
        end
    end
end

Swimmer Augustyn, Kate assigned to event 200 Back (Best time: 123.96)
Swimmer Bernard, Sarah assigned to event 200 Breast (Best time: 136.67)
Swimmer Naveen, Annika assigned to event 50 Free (Best time: 23.53)
Swimmer Roberson, Ella assigned to event 1000 Free (Best time: 624.84)
Swimmer Roberson, Ella assigned to event 500 Free (Best time: 303.66)
Swimmer Roberson, Ella assigned to event 200 IM (Best time: 125.25)
Swimmer Simons, Kailey assigned to event 100 Breast (Best time: 60.29)
Swimmer Smith, Sydney assigned to event 200 Free (Best time: 116.24)
Swimmer Smith, Sydney assigned to event 100 Free (Best time: 52.48)
Swimmer Smith, Sydney assigned to event 100 Fly (Best time: 57.21)
Swimmer Swartwood, Belise assigned to event 200 Fly (Best time: 125.71)
Swimmer Yang, Iris assigned to event 100 Back (Best time: 55.8)


In [8]:
#prints out each relay group, the position (SHOULD BE 1-4), the swimmer, and the time 
for r in 1:num_relay_events
    for g in 1:num_groups
        println("Relay $(r), Group $(g):")
        for i in 1:num_relay_positions
            for s in 1:swimmer_count
                if value(y[s, i, r, g]) > 0.5
                    swimmer = swimmer_list[s]
                    event = relay_event_order[r, "swimmer_$(i)_event"]
                    best_time = get(best_times, (swimmer, event), "N/A")
                    println("  Position $(i): $(swimmer) (Best time: $(best_time))")
                end
            end
        end
    end
end

Relay 1, Group 1:
  Position 1: Smith, Sydney (Best time: 27.47)
  Position 2: Simons, Kailey (Best time: 30.34)
  Position 3: Beggs, Christina (Best time: 20.09)
  Position 4: Chen, Olivia (Best time: 24.48)
Relay 1, Group 2:
  Position 1: Augustyn, Kate (Best time: 26.5)
  Position 2: Bernard, Sarah (Best time: 29.97)
  Position 3: Naveen, Annika (Best time: 24.89)
  Position 4: Turvey, Alexandra (Best time: 23.61)
Relay 1, Group 3:
  Position 1: Yang, Iris (Best time: 26.18)
  Position 2: Li, Anna (Best time: 30.55)
  Position 3: Zhao, Kathy (Best time: 25.44)
  Position 4: Poag, Lillian (Best time: 24.25)
Relay 2, Group 1:
  Position 1: Smith, Sydney (Best time: 24.85)
  Position 2: Chen, Olivia (Best time: 24.48)
  Position 3: Beggs, Christina (Best time: 24.12)
  Position 4: Naveen, Annika (Best time: 23.53)
Relay 2, Group 2:
  Position 1: Augustyn, Kate (Best time: 24.88)
  Position 2: Kudela, Katie (Best time: 24.87)
  Position 3: Simons, Kailey (Best time: 24.55)
  Position 4:

In [9]:
#prints out the number of events swimmers are in, both individual and relay
for s in 1:swimmer_count
    individual_events = sum(value(x[s, e]) for e in 1:num_individual_events)
    relay_events = sum(value(y[s, i, r, g]) for i in 1:num_relay_positions, r in 1:num_relay_events, g in 1:num_groups)
    total_events = individual_events + relay_events
    println("Swimmer $(swimmer_list[s]): $(total_events) events ($(individual_events) individual, $(relay_events) relay)")
end

Swimmer Adler, Lauren: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Augustyn, Kate: 3.0 events (1.0 individual, 2.0 relay)
Swimmer Beggs, Christina: 2.0 events (-0.0 individual, 2.0 relay)
Swimmer Bernard, Sarah: 2.0 events (1.0 individual, 1.0 relay)
Swimmer Chen, Olivia: 2.0 events (-0.0 individual, 2.0 relay)
Swimmer Chun, Sydney: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Crane, Jessie: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Eppinger, Aria: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Feliz, Mary: 1.0 events (-0.0 individual, 1.0 relay)
Swimmer Griffin, Miri: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Kim, Jolie: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Kudela, Katie: 1.0 events (-0.0 individual, 1.0 relay)
Swimmer Levy, Lauren: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Li, Anna: 1.0 events (-0.0 individual, 1.0 relay)
Swimmer Lu, Sarah: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Naveen, Annika: 3.0 events (1.0 individual, 

### Model 2: Maximizing Total Points

In [10]:
# Points vectors
p_i = [9, 4, 3, 2, 1, 0]  # Points for individual events
p_r = [11, 4, 2, 0, 0, 0]  # Points for relay events

6-element Vector{Int64}:
 11
  4
  2
  0
  0
  0

In [11]:
num_places = 6 # 6 different places to finish a race
num_opponent = 21

21

In [12]:
opponent_list = unique(opponent_times[:, :Opponent])
opponent_time_dict = Dict((row[:Event], row[:Opponent]) => row[:Opponent_Time] for row in eachrow(opponent_times))

Dict{Tuple{String31, String31}, Float64} with 41 entries:
  ("50 Free", "Brenner, Mandy")           => 24.15
  ("200 Back", "Brock, Lizzie")           => 126.95
  ("400 Free Relay", "Group 2")           => 216.17
  ("100 Breast", "Augustyn, Gabi")        => 63.68
  ("100 Breast", "Denisenko, Aleksandra") => 65.21
  ("100 Back", "Wieclawek, Blythe")       => 56.6
  ("100 Free", "Brenner, Mandy")          => 52.89
  ("1000 Free", "Cruz-Abrams, Robin")     => 653.79
  ("200 Fly", "Yoon, Grace")              => 125.89
  ("1000 Free", "Wood Prince, Piper")     => 622.34
  ("100 Back", "Hamlin, Molly")           => 54.94
  ("500 Free", "Bastone, Alexandra")      => 291.62
  ("200 IM", "Iannaccone, Stephanie")     => 123.86
  ("100 Back", "Wilhelm, Kiley")          => 58.88
  ("50 Free", "Alas, Isabella")           => 24.02
  ("200 Breast", "Iannaccone, Stephanie") => 136.48
  ("200 Free", "Hakonardottir, Kristin")  => 116.2
  ("50 Free", "Hakonardottir, Kristin")   => 24.75
  ("1000 Free", "

In [23]:
# Create model
model_2 = Model(Gurobi.Optimizer)

# Variables
@variable(model_2, x[1:swimmer_count, 1:num_individual_events, 1:num_places], Bin)
@variable(model_2, y[1:swimmer_count, 1:num_relay_positions, 1:num_relay_events, 1:num_groups], Bin)
@variable(model_2, z_iok[1:num_individual_events, 1:num_opponent, 1:num_places], Bin)
@variable(model_2, z_gk[1:num_groups, 1:num_places], Bin)
@variable(model_2, S_mk[1:3, 1:num_places], Bin)

# Objective function
@objective(model_2, Max, 
    sum(p_i[k] * x[s, i, k] for s in 1:swimmer_count, i in 1:num_individual_events, k in 1:num_places) +
    sum(p_r[k] * z_gk[g, k] for g in 1:num_groups, k in 1:num_places)
)

# Constraints
# Each swimmer swims max 3 individual events
@constraint(model_2, [s in 1:swimmer_count], 
    sum(x[s, i, k] for i in 1:num_individual_events, k in 1:num_places) <= 3)

# 4 persons per relay group
@constraint(model_2, [r in 1:num_relay_events, g in 1:num_groups], 
    sum(y[s, i, r, g] for s in 1:swimmer_count, i in 1:num_relay_positions) == 4)

# Can't swim back-to-back events
for s in 1:swimmer_count
    for i in 1:num_individual_events - 1
        @constraint(model_2, sum(x[s, i, k] + x[s, i + 1, k] for k in 1:num_places) <= 1)
    end
end

# Each swimmer can only swim once in first relay
@constraint(model_2, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# Each swimmer can only swim once in last relay
@constraint(model_2, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# Each swimmer can only swim in one position in a group
@constraint(model_2, [r in 1:num_relay_events, g in 1:num_groups, i in 1:num_relay_positions], 
    sum(y[s, i, r, g] for s in 1:swimmer_count) == 1)

# If swim in first relay, can't swim first individual event
@constraint(model_2, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) + sum(x[s, 1, k] for k in 1:num_places) <= 1)

# If swim in last relay, can't swim last individual event
@constraint(model_2, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) + sum(x[s, num_individual_events, k] for k in 1:num_places) <= 1)

# Placing constraints for individual events
#@constraint(model_2, [i in 1:num_individual_events, o in 1:num_opponent, k in 1:(num_places - 1), s in 1:swimmer_count],
#    x[s, i, k] * get(mean_times, (swimmer_list[s], individual_event_order_list[i]), penalty_time) <= 
 #   z_iok[i, o, k] * get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), penalty_time))

#@constraint(model_2, [i in 1:num_individual_events, o in 1:num_opponent, k in 1:(num_places - 1)],
 #   z_iok[i, o, k] * get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), penalty_time) <= 
  #  z_iok[i, o, k + 1] * get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), penalty_time))

# Placing constraints for relay events
#@constraint(model_2, [g in 1:num_groups, k in 1:(num_places - 1)],
 #   z_gk[g, k] * sum(get(mean_times, (swimmer_list[s], relay_event_positions[relay_event_order[r, 2]][i]), penalty_time) * y[s, i, r, g] 
  #      for s in 1:swimmer_count, i in 1:num_relay_positions, r in 1:num_relay_events) 
   # <= S_mk[g, k + 1] * sum(get(opponent_time_dict, (relay_event_order[r, 2], opponent_list[m]), penalty_time) for r in 1:num_relay_events, m in 1:num_opponent))

# Only one swimmer/opponent per place per event
@constraint(model_2, [i in 1:num_individual_events, k in 1:num_places],
    sum(x[s, i, k] for s in 1:swimmer_count) + sum(z_iok[i, o, k] for o in 1:num_opponent) == 1)

# Each relay group gets only one place
@constraint(model_2, [g in 1:num_groups],
    sum(z_gk[g, k] for k in 1:num_places) <= 1)

# Each opponent relay group gets only one place
@constraint(model_2, [m in 1:3],
    sum(S_mk[m, k] for k in 1:num_places) <= 1)

# Solve the model
optimize!(model_2)

if termination_status(model_2) == MOI.OPTIMAL
    println("Optimal solution found")
else
    println("No optimal solution found")
end

println("Total points won: ", objective_value(model_2))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-26
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 604 rows, 4524 columns and 13452 nonzeros
Model fingerprint: 0xde4f63e9
Variable types: 0 continuous, 4524 integer (4524 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 33.0000000
Presolve removed 86 rows and 1920 columns
Presolve time: 0.02s
Presolved: 518 rows, 2604 columns, 8928 nonzeros
Variable types: 0 continuous, 2604 integer (2604 binary)
Found heuristic solution: objective 261.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 

In [24]:
if termination_status(model_2) == MOI.OPTIMAL
    println("Optimal solution found")
    println("Total points won: ", objective_value(model_2))
    
    # Individual events points
    println("\nIndividual Events Points:")
    for s in 1:swimmer_count
        for i in 1:num_individual_events
            for k in 1:num_places
                if value(x[s, i, k]) > 1e-6
                    swimmer = swimmer_list[s]
                    event = individual_event_order_list[i]
                    place = k
                    points = p_i[k]
                    println("Swimmer: $swimmer, Event: $event, Place: $place, Points: $points")
                end
            end
        end
    end
else
    println("No optimal solution found")
end


Optimal solution found
Total points won: 261.0

Individual Events Points:
Swimmer: Adler, Lauren, Event: 1000 Free, Place: 1, Points: 9
Swimmer: Adler, Lauren, Event: 100 Back, Place: 1, Points: 9
Swimmer: Adler, Lauren, Event: 200 Fly, Place: 1, Points: 9
Swimmer: Augustyn, Kate, Event: 200 Free, Place: 1, Points: 9
Swimmer: Augustyn, Kate, Event: 100 Breast, Place: 1, Points: 9
Swimmer: Augustyn, Kate, Event: 50 Free, Place: 1, Points: 9
Swimmer: Beggs, Christina, Event: 100 Free, Place: 1, Points: 9
Swimmer: Beggs, Christina, Event: 200 Breast, Place: 1, Points: 9
Swimmer: Beggs, Christina, Event: 100 Fly, Place: 1, Points: 9
Swimmer: Bernard, Sarah, Event: 200 Back, Place: 1, Points: 9
Swimmer: Bernard, Sarah, Event: 500 Free, Place: 1, Points: 9
Swimmer: Bernard, Sarah, Event: 200 IM, Place: 1, Points: 9
Swimmer: Chen, Olivia, Event: 200 Free, Place: 2, Points: 4
Swimmer: Chen, Olivia, Event: 100 Breast, Place: 2, Points: 4
Swimmer: Chen, Olivia, Event: 50 Free, Place: 2, Points: 

In [25]:
# Individual events points
println("\nIndividual Events Points:")
for s in 1:swimmer_count
    for i in 1:num_individual_events
        for k in 1:num_places
            if value(x[s, i, k]) > 0.5
                swimmer = swimmer_list[s]
                event = individual_event_order_list[i]
                place = k
                points = p_i[k]
                println("Swimmer: $swimmer, Event: $event, Place: $place, Points: $points")
            end
        end
    end
end


Individual Events Points:
Swimmer: Adler, Lauren, Event: 1000 Free, Place: 1, Points: 9
Swimmer: Adler, Lauren, Event: 100 Back, Place: 1, Points: 9
Swimmer: Adler, Lauren, Event: 200 Fly, Place: 1, Points: 9
Swimmer: Augustyn, Kate, Event: 200 Free, Place: 1, Points: 9
Swimmer: Augustyn, Kate, Event: 100 Breast, Place: 1, Points: 9
Swimmer: Augustyn, Kate, Event: 50 Free, Place: 1, Points: 9
Swimmer: Beggs, Christina, Event: 100 Free, Place: 1, Points: 9
Swimmer: Beggs, Christina, Event: 200 Breast, Place: 1, Points: 9
Swimmer: Beggs, Christina, Event: 100 Fly, Place: 1, Points: 9
Swimmer: Bernard, Sarah, Event: 200 Back, Place: 1, Points: 9
Swimmer: Bernard, Sarah, Event: 500 Free, Place: 1, Points: 9
Swimmer: Bernard, Sarah, Event: 200 IM, Place: 1, Points: 9
Swimmer: Chen, Olivia, Event: 200 Free, Place: 2, Points: 4
Swimmer: Chen, Olivia, Event: 100 Breast, Place: 2, Points: 4
Swimmer: Chen, Olivia, Event: 50 Free, Place: 2, Points: 4
Swimmer: Chun, Sydney, Event: 1000 Free, Plac

In [26]:
println("\nRelay Events Points and Assignments:")
for r in 1:num_relay_events
    for g in 1:num_groups
        # Check if this group got points
        for k in 1:num_places
            if value(z_gk[g, k]) > 0.5
                points = p_r[k]
                println("Relay: $r Group: $g Place: $k Points: $points")
            end
        end
        
        # Print relay assignments
        for k in 1:num_places
            if value(z_gk[g, k]) > 0.5
                println("Relay Group $g, Place: $k, Points: $(p_r[k])")
                for i in 1:num_relay_positions
                    for s in 1:swimmer_count
                        if value(y[s, i, r, g]) > 0.5
                            swimmer = swimmer_list[s]
                            event = relay_event_order[r, :event]
                            best_time = get(best_times, (swimmer, event), "N/A")
                            println("  Swimmer: $swimmer, Position: $i, Event: $event, Best time: $best_time")
                        end
                    end
                end
            end
        end
    end
end


Relay Events Points and Assignments:
Relay: 1 Group: 1 Place: 1 Points: 11
Relay Group 1, Place: 1, Points: 11
  Swimmer: Shipps, Stella, Position: 1, Event: 200 Medley Relay, Best time: N/A
  Swimmer: Zhao, Kathy, Position: 2, Event: 200 Medley Relay, Best time: N/A
  Swimmer: Bernard, Sarah, Position: 3, Event: 200 Medley Relay, Best time: N/A
  Swimmer: Yang, Iris, Position: 4, Event: 200 Medley Relay, Best time: N/A
Relay: 1 Group: 2 Place: 1 Points: 11
Relay Group 2, Place: 1, Points: 11
  Swimmer: Chen, Olivia, Position: 1, Event: 200 Medley Relay, Best time: N/A
  Swimmer: Kudela, Katie, Position: 2, Event: 200 Medley Relay, Best time: N/A
  Swimmer: Poag, Lillian, Position: 3, Event: 200 Medley Relay, Best time: N/A
  Swimmer: Feliz, Mary, Position: 4, Event: 200 Medley Relay, Best time: N/A
Relay: 1 Group: 3 Place: 1 Points: 11
Relay Group 3, Place: 1, Points: 11
  Swimmer: Kim, Jolie, Position: 1, Event: 200 Medley Relay, Best time: N/A
  Swimmer: Lu, Sarah, Position: 2, Eve

In [27]:
# Calculate points for opponent in individual events
opponent_individual_points = 0
for i in 1:num_individual_events
    for k in 1:num_places
        for o in 1:num_opponent
        if value(z_iok[i,o,k]) > 0.5  # Check if the opponent is assigned to place k in event i
            opponent_individual_points += p_i[k]
            end
        end
    end
end

# Calculate points for opponent in relay events
opponent_relay_points = 0
for m in 1:3  # Iterate over opponent relay groups
    for k in 1:num_places
        if value(S_mk[m,k]) > 0.5  # Check if the opponent relay group m is assigned to place k
            opponent_relay_points += p_r[k]
        end
    end
end

# Total opponent points
total_opponent_points = opponent_individual_points + opponent_relay_points

# Print results
println("Opponent points from individual events: ", opponent_individual_points)
println("Opponent points from relay events: ", opponent_relay_points)
println("Total opponent points: ", total_opponent_points)

Opponent points from individual events: 0
Opponent points from relay events: 0
Total opponent points: 0


In [17]:
total_points = sum(p_i[k] * value(z_isk[s,i,k]) for s in 1:swimmer_count, i in 1:num_individual_events, k in 1:num_places) +
               sum(p_i[k] * value(z_iok[i,k]) for i in 1:num_individual_events, k in 1:num_places) +
               sum(p_r[k] * value(z_gk[g,k]) for g in 1:num_groups, k in 1:num_places) +
               sum(p_r[k] * value(S_mk[m,k]) for m in 1:3, k in 1:num_places)
println("Total calculated points: ", total_points)
println("Expected total points: 300")

LoadError: UndefVarError: `z_isk` not defined